# 背景：
    整理单目录下10W+的照片文件，名称YYYYMMDD_IMG_xxxx.(jpg\png\heic)
# 需求：
    1.将jpg\png以外格式的文件，移入other
    2.根据文件名YYYYMMDD，移入对应日期的文件夹(例如:202107)
    3.去除文件名中的日期(YYYYMMDD_)

In [14]:
import os, shutil

PATH = r"\\192.168.31.99\photo\hanminmin\相机胶卷"
FILE_LIST = ['.JPG','.PNG']

def check_create_dir(dir_path):
    print('检查创建文件夹:' + dir_path)
    fd = os.path.exists(dir_path)
    if not fd:
        print('创建目录：%s' % dir_path)
        os.makedirs(dir_path)
    else:
        print('目录已存在：%s' % dir_path)

def move_file(file_path, dir_path):
    # 校验目标目录是否存在，不存在则新建
    check_create_dir(dir_path)
    # 校验目标目录下是否已存在该文件
    file_name = os.path.basename(file_path)
    fd = os.path.exists(os.path.join(dir_path, file_name))
    if not fd:
        print('移动文件：%s' % file_path)
        shutil.move(file_path, dir_path)
    else:
        print('文件已存在：%s' % file_path)

def do_file(file, current_dir_path):
    file_path = os.path.join(current_dir_path, file)
    file_split = os.path.splitext(file)
    file_type = file_split[1]
    if(file_type in FILE_LIST):
        file_name = str(file_split[0])
        date_dir_name = file_name.split('_')[0]
        date_dir_path = os.path.join(current_dir_path, date_dir_name[0:6])
        move_file(file_path, date_dir_path)
    else:
        other_dir_path = os.path.join(current_dir_path, 'other')
        move_file(file_path, other_dir_path)


## 第一步
### 将指定文件夹下的文件整理到对应的日期文件夹下
1.单线程串行

In [15]:
def move_my_file(work_path):
    list = os.listdir(work_path)
    print(len(list))
    i = 0
    for file in list:
        file_path = os.path.join(work_path, file)
        if os.path.isfile(file_path):
            i += 1
            do_file(file, work_path)

    print('success!%d条数据移动成功' % i)

# move_my_file(PATH)

2.多线程并行

In [16]:
import queue
import threading
import _thread
import time

# 创建一个队列对象，把数组值放进去
my_queue = queue.Queue(maxsize=-1)

def build_queue_list(work_path):
    print("=========启动一个thread-queue")
    file_list = os.listdir(work_path)
    print(len(file_list))
    i = 1
    for file in file_list:
        file_path = os.path.join(work_path, file)
        if os.path.isfile(file_path):
            i += 1
            print("队列中添加一条数据:%d" % i)
            list = [file, work_path]
            global my_queue
            my_queue.put(list)
    print("列表中的数据个数%s" % my_queue.qsize())
    print("=========结束线程thread-queue")

# 从队列中取出值，并调用实际操作
def customer(thread_name):
    print("=========启动一个thread-%d" % thread_name)
    while not my_queue.empty():
        list = my_queue.get()
        print("thread-%d开始执行do_file()方法,入参file:%s,dir:%s=====" % (thread_name, list[0], list[1]))
        do_file(list[0], list[1])
    print("=========结束线程thread-%d" % thread_name)

def _thread_move_my_file(work_path, thread_count):
    print("添加一个线程，执行队列操作")
    _thread.start_new_thread(build_queue_list, (work_path,))
    print("开始等待")
    time.sleep(1)
    print("结束等待")
    for t in range(thread_count):
        print('添加了一个线程%d' % t)
        _thread.start_new_thread(customer, (t,))
    print("success")

def thread_move_my_file(work_path, thread_count):
    print("=========================开始")
    # 先启动获取队列方法
    thread_queue = threading.Thread(target=build_queue_list, args=(work_path,))

    # 起10个线程，线程target去执行从队列中取值并进行操作的动作
    threads = []
    threads.append(thread_queue)
    print("=========================启动queue线程")
    thread_queue.start()
    print("开始等待")
    time.sleep(15)
    print("结束等待")
    print("=========================添加线程池")
    for t in range(thread_count):
        thread = threading.Thread(target=customer, args=(t,))
        threads.append(thread)
        thread.start()

    print("=========================启动线程池")
    for t in threads:
        t.join()
    print("success")

# _thread_move_my_file(PATH, 10)


第二步
遍历所有文件并修改文件名(去除日期YYYYMMDD_开头)

In [24]:
def rename_my_file(my_file_path):
    for root, dirs, files in os.walk(my_file_path):
        print('root:')
        print(root)  # 当前目录路径
        print('dirs:')
        print(dirs)  # 当前路径下所有子目录
        print('files:')
        # print(files)  # 当前路径下所有非目录子文件
        print('========================')
        for file in files:
            print(file)
            if file[0:3] == 'IMG':
                continue
            else:
                file_path = os.path.join(root, file)
                print('file_path:'+file_path)
                new_file_name = file[9:]
                print('new_file_name:'+new_file_name)
                new_file_path = os.path.join(root, new_file_name)
                fd = os.path.exists(new_file_path)
                print('fd=='+str(fd))
                if not fd:
                    pass
                else:
                    # 如果存在，则在文件名后加"_1"，方便区分
                    file_name_splitext = os.path.splitext(new_file_name)
                    new_file_name = file_name_splitext[0] + '_1' + file_name_splitext[1]
                    new_file_path = os.path.join(root, new_file_name)
                os.rename(file_path, new_file_path)

    print('success')

# rename_my_file(PATH)

In [25]:
my_file_path = r"\\192.168.31.99\photo\hanminmin\photo\相机胶卷"
# move_my_file(my_file_path)
# thread_move_my_file(my_file_path, 10)
my_file_path = r'\\192.168.31.99\photo\hanminmin\photo\相机胶卷\202103'
rename_my_file(my_file_path)

root:
\\192.168.31.99\photo\hanminmin\photo\相机胶卷\202103
dirs:
[]
files:
IMG_5598.JPG
success
